In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")


from dotenv import load_dotenv
import os

env_path = '../../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
AQI_API_KEY = os.getenv("AQI_API_KEY")

In [2]:
csv_file="../../data/jing'an-monitoring station-air-quality.csv"
util.check_file_path(csv_file)

File successfully found at the path: ../../data/jing'an-monitoring station-air-quality.csv


In [3]:
country="china"
city = "shanghai"
street = "jinganjiancezhan"
aqicn_url="https://api.waqi.info/feed/@486"

latitude, longitude = util.get_city_coordinates(city)
today = datetime.date.today()
latitude, longitude, today

(31.23, 121.47, datetime.date(2024, 11, 10))

In [4]:
project = hopsworks.login(project='AMomozZz')

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163416


In [5]:
secrets = util.secrets_api(project.name)
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,114.0,china,shanghai,jinganjiancezhan,2024-11-10,https://api.waqi.info/feed/@486


In [7]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

,date,pm25,pm10,o3,no2,so2,co
0,2024-11-01,20.0,45.0,44.0,13.0,3.0,5.0
1,2024-11-02,103.0,58.0,47.0,19.0,3.0,6.0
2,2024-11-03,140.0,48.0,58.0,18.0,4.0,6.0
3,2024-11-04,124.0,21.0,31.0,10.0,3.0,3.0
4,2024-11-05,45.0,20.0,35.0,12.0,3.0,3.0
...,...,...,...,...,...,...,...
3954,2018-12-31,NaN,33.0,27.0,10.0,4.0,5.0
3955,2017-09-10,NaN,31.0,28.0,18.0,4.0,11.0
3956,2016-01-03,NaN,95.0,30.0,35.0,11.0,10.0
3957,2014-12-31,NaN,61.0,21.0,21.0,16.0,6.0


In [8]:
# These commands will succeed if your CSV file didn't have a `median` or `timestamp` column
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')
df_aq

,date,pm25
0,2024-11-01,20.0
1,2024-11-02,103.0
2,2024-11-03,140.0
3,2024-11-04,124.0
4,2024-11-05,45.0
...,...,...
3954,2018-12-31,NaN
3955,2017-09-10,NaN
3956,2016-01-03,NaN
3957,2014-12-31,NaN


In [9]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3959 entries, 0 to 3958
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3959 non-null   datetime64[ns]
 1   pm25    3952 non-null   float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 46.5 KB


In [10]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25
0,2024-11-01,20.0
1,2024-11-02,103.0
2,2024-11-03,140.0
3,2024-11-04,124.0
4,2024-11-05,45.0
...,...,...
3947,2014-03-27,108.0
3948,2014-03-28,103.0
3949,2014-03-29,153.0
3950,2014-03-30,173.0


In [11]:
# Your sensor may have columns we won't use, so only keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url

df_aq.sort_values(by='date', ascending=True).tail(10)

,date,pm25,country,city,street,url
0,2024-11-01,20.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1,2024-11-02,103.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2,2024-11-03,140.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3,2024-11-04,124.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
4,2024-11-05,45.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
5,2024-11-06,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
6,2024-11-07,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
7,2024-11-08,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
8,2024-11-09,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
9,2024-11-10,56.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


In [12]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

# new_row_df = pd.DataFrame([{'date': pd.to_datetime('2024-11-09'), 'temperature_2m_mean': 17.700001, 'precipitation_sum': 0.0, 'wind_speed_10m_max': 13.513193, 'wind_direction_10m_dominant': 41.760216, 'city': 'shanghai'}])
# new_row_df['temperature_2m_mean'] = new_row_df['temperature_2m_mean'].astype('float32')
# new_row_df['precipitation_sum'] = new_row_df['precipitation_sum'].astype('float32')
# new_row_df['wind_speed_10m_max'] = new_row_df['wind_speed_10m_max'].astype('float32')
# new_row_df['wind_direction_10m_dominant'] = new_row_df['wind_direction_10m_dominant'].astype('float32')
# weather_df = pd.concat([weather_df, new_row_df], ignore_index=True)

df_temp = df_aq.copy()
df_temp['date'] = pd.to_datetime(df_temp['date']) + pd.Timedelta(days=2)

weather_df = pd.merge(weather_df, df_temp[['date','pm25']], how='left', on="date")
weather_df = weather_df.rename(columns={"pm25": "pm25_tdby"})
weather_df.dropna(inplace=True)

weather_df.sort_values(by='date', ascending=True).tail(10)

Coordinates 31.247802734375°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_tdby
3955,2024-10-30,19.104918,0.000000,23.185787,42.530087,shanghai,115.0
3956,2024-10-31,21.023664,72.799995,24.979176,44.617844,shanghai,49.0
3957,2024-11-01,18.234087,58.000004,41.148014,1.924738,shanghai,54.0
3958,2024-11-02,17.463247,0.000000,24.215185,335.282562,shanghai,49.0
3959,2024-11-03,17.740332,0.000000,8.149847,16.130766,shanghai,20.0
3960,2024-11-04,17.409084,0.000000,19.645580,10.283305,shanghai,103.0
3961,2024-11-05,13.817414,0.000000,19.852295,358.205353,shanghai,140.0
3962,2024-11-06,13.725750,0.000000,17.589418,15.969136,shanghai,124.0
3963,2024-11-07,14.234082,0.000000,14.450535,34.656887,shanghai,45.0
3964,2024-11-08,17.711164,0.000000,18.275984,44.213470,shanghai,35.0


In [13]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3948 entries, 2 to 3964
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3948 non-null   datetime64[ns]
 1   temperature_2m_mean          3948 non-null   float32       
 2   precipitation_sum            3948 non-null   float32       
 3   wind_speed_10m_max           3948 non-null   float32       
 4   wind_direction_10m_dominant  3948 non-null   float32       
 5   city                         3948 non-null   object        
 6   pm25_tdby                    3948 non-null   float32       
dtypes: datetime64[ns](1), float32(5), object(1)
memory usage: 169.6+ KB


In [14]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [15]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")
expect_greater_than_zero("pm25_tdby")

In [16]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "china", "city": "shanghai", "street": "jinganjiancezhan", "aqicn_url": "https://api.waqi.info/feed/@486", "latitude": 31.23, "longitude": 121.47}


In [18]:
air_quality_fg = fs.get_or_create_feature_group(
    name='new_aq',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

In [19]:
_ = air_quality_fg.insert(df_aq)

2024-11-10 20:24:33,253 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1344865


Uploading Dataframe: 0.00% |          | Rows 0/3952 | Elapsed Time: 00:00 | Remaining Time: ?

KafkaException: KafkaError{code=_TIMED_OUT,val=-185,str="Failed to get metadata: Local: Timed out"}

In [ ]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [ ]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='new_w',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

In [ ]:
# Insert data
_ = weather_fg.insert(weather_df)

2024-11-10 20:19:15,140 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1344864


Uploading Dataframe: 0.00% |          | Rows 0/3948 | Elapsed Time: 00:00 | Remaining Time: ?

KafkaException: KafkaError{code=_TIMED_OUT,val=-185,str="Failed to get metadata: Local: Timed out"}

In [ ]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")
weather_fg.update_feature_description("pm25_tdby", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [ ]:
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(10))
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.12s) 


,date,pm25,country,city,street,url
3791,2024-11-01 00:00:00+00:00,20.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3722,2024-11-02 00:00:00+00:00,103.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
708,2024-11-03 00:00:00+00:00,140.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
540,2024-11-04 00:00:00+00:00,124.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
348,2024-11-05 00:00:00+00:00,45.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1286,2024-11-06 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1370,2024-11-07 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3121,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2555,2024-11-09 00:00:00+00:00,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3039,2024-11-10 00:00:00+00:00,56.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_tdby
1453,2024-10-30 00:00:00+00:00,19.104918,0.000000,23.185787,42.530087,shanghai,115.0
285,2024-10-31 00:00:00+00:00,21.023664,72.799995,24.979176,44.617844,shanghai,49.0
2249,2024-11-01 00:00:00+00:00,18.234087,58.000004,41.148014,1.924738,shanghai,54.0
431,2024-11-02 00:00:00+00:00,17.463247,0.000000,24.215185,335.282562,shanghai,49.0
2330,2024-11-03 00:00:00+00:00,17.740332,0.000000,8.149847,16.130766,shanghai,20.0
3795,2024-11-04 00:00:00+00:00,17.409084,0.000000,19.645580,10.283305,shanghai,103.0
2921,2024-11-05 00:00:00+00:00,13.817414,0.000000,19.852295,358.205353,shanghai,140.0
1253,2024-11-06 00:00:00+00:00,13.725750,0.000000,17.589418,15.969136,shanghai,124.0
2532,2024-11-07 00:00:00+00:00,14.234082,0.000000,14.450535,34.656887,shanghai,45.0
2318,2024-11-08 00:00:00+00:00,17.711164,0.000000,18.275984,44.213470,shanghai,35.0
